#Connect Gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Install

In [ ]:
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 1.8 MB/s eta 0:00:00


In [ ]:
!pip install translate

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#Library

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib as plt
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from translate import Translator
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize


#Data Cleaning

In [ ]:
#Load Data
data = pd.read_csv('/content/drive/MyDrive/Project1/JulClothing/Data Mentah Jul.csv')
data

,Produk,Nama,Waktu,Review,Rating
0,Kaos polo polos / Kaos kerah / Kaos wangki,K***i,1 hari lalu,Ukurannya pas Tinggi badan 160 - 170 cm,bintang 5
1,Kaos pria o neck need coffe,Veronica,1 hari lalu,"Sesuai foto, cuma tipis aja.. kl difoto kliata...",bintang 5
2,Croptop Wanita girl - Lengan Pendek Cotton Com...,FVou,2 hari lalu,155/48. I love it!,bintang 5
3,Kaos Polos Wanita Cotton Combed 30s,A***i,2 hari lalu,Ukurannya pas Tinggi badan 161 - 170 cm Berat ...,bintang 5
4,Kaos Polos Wanita Cotton Combed 30s,nia,2 hari lalu,Ukurannya pas,bintang 5
...,...,...,...,...,...
372,KAOS POLO COMBED,iwan,9 bulan lalu,"barang nya bagus, sablon ny keren bahan nya ad...",bintang 5
373,KAOS 78th DIRGAHAYU REPUBLIK INDONESIA / KAOS ...,alfira,9 bulan lalu,Ukurannya pas,bintang 5
374,KAOS 78th DIRGAHAYU REPUBLIK INDONESIA / KAOS ...,Yulina,9 bulan lalu,terima kasih 🙏\n\nterima kasih 🙏,bintang 5
375,Kaos wanita friends - Cotton Combed 30s,Meutia,9 bulan lalu,Bajunya pas dan nggak nerawang. Untuk harga se...,bintang 5


In [ ]:
#Menghapus string bintang pada kolom Rating
hapusString = data.replace(to_replace='bintang', value="", regex=True)

In [ ]:
#Membuat csv baru
hapusString.to_csv('/content/drive/MyDrive/Project1/JulClothing/Tokped-JulCleaned.csv', index = False)

In [ ]:
#Load data baru
df = pd.read_csv('/content/drive/MyDrive/Project1/JulClothing/Tokped-JulCleaned.csv')
df

,Produk,Nama,Waktu,Review,Rating
0,Kaos polo polos / Kaos kerah / Kaos wangki,K***i,1 hari lalu,Ukurannya pas Tinggi badan 160 - 170 cm,5
1,Kaos pria o neck need coffe,Veronica,1 hari lalu,"Sesuai foto, cuma tipis aja.. kl difoto kliata...",5
2,Croptop Wanita girl - Lengan Pendek Cotton Com...,FVou,2 hari lalu,155/48. I love it!,5
3,Kaos Polos Wanita Cotton Combed 30s,A***i,2 hari lalu,Ukurannya pas Tinggi badan 161 - 170 cm Berat ...,5
4,Kaos Polos Wanita Cotton Combed 30s,nia,2 hari lalu,Ukurannya pas,5
...,...,...,...,...,...
372,KAOS POLO COMBED,iwan,9 bulan lalu,"barang nya bagus, sablon ny keren bahan nya ad...",5
373,KAOS 78th DIRGAHAYU REPUBLIK INDONESIA / KAOS ...,alfira,9 bulan lalu,Ukurannya pas,5
374,KAOS 78th DIRGAHAYU REPUBLIK INDONESIA / KAOS ...,Yulina,9 bulan lalu,terima kasih 🙏\n\nterima kasih 🙏,5
375,Kaos wanita friends - Cotton Combed 30s,Meutia,9 bulan lalu,Bajunya pas dan nggak nerawang. Untuk harga se...,5


In [ ]:
#semua string di kolom review menjadi huruf kecil
df['Review'] = df['Review'].str.lower()

In [ ]:
  #Menghilangkan emoji
  def remove_emojis(df):
      emoj = re.compile("["
          u"\U00002700-\U000027BF"  # Dingbats
          u"\U0001F600-\U0001F64F"  # Emoticons
          u"\U00002600-\U000026FF"  # Miscellaneous Symbols
          u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols And Pictographs
          u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
          u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
          u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
                        "]+", re.UNICODE)
      return re.sub(emoj, '', df)
      return remove_emojis(value)

  df['Review'] = df['Review'].apply(remove_emojis)

In [ ]:
#Melihat berapa baris dan kolom
df.shape

(377, 5)

In [ ]:
#Mencari data duplikat
df.duplicated().sum()

2

In [ ]:
df

,Produk,Nama,Waktu,Review,Rating
0,Kaos polo polos / Kaos kerah / Kaos wangki,K***i,1 hari lalu,ukurannya pas tinggi badan 160 - 170 cm,5
1,Kaos pria o neck need coffe,Veronica,1 hari lalu,"sesuai foto, cuma tipis aja.. kl difoto kliata...",5
2,Croptop Wanita girl - Lengan Pendek Cotton Com...,FVou,2 hari lalu,155/48. i love it!,5
3,Kaos Polos Wanita Cotton Combed 30s,A***i,2 hari lalu,ukurannya pas tinggi badan 161 - 170 cm berat ...,5
4,Kaos Polos Wanita Cotton Combed 30s,nia,2 hari lalu,ukurannya pas,5
...,...,...,...,...,...
372,KAOS POLO COMBED,iwan,9 bulan lalu,"barang nya bagus, sablon ny keren bahan nya ad...",5
373,KAOS 78th DIRGAHAYU REPUBLIK INDONESIA / KAOS ...,alfira,9 bulan lalu,ukurannya pas,5
374,KAOS 78th DIRGAHAYU REPUBLIK INDONESIA / KAOS ...,Yulina,9 bulan lalu,terima kasih \n\nterima kasih,5
375,Kaos wanita friends - Cotton Combed 30s,Meutia,9 bulan lalu,bajunya pas dan nggak nerawang. untuk harga se...,5


#Data Preprocessing

In [ ]:
#Normalisasi

norm = {"yg" : "yang",
        "udh" : "udah",
        "bgt" : "banget",
        "hr" : "hari",
        "tks" : "thanks",
        "thx" : "thanks",
        "unt" : "untuk",
        "cs" : "customer service",
        "pls" : "please",
        "gk" : "gak",
        "dpt" : "dapet",
        "gpp" : "tidak apa-apa",
        "dg" : "dengan",
        "bg" : "bang",
        "spt" : "seperti",
        "gt" : "gitu",
        "mntp" : "mantap",
        "bhn" : "bahan",
        "krn" : "keren",
        "jg" : "juga",
        "goks" : "gokil"
        }

def normalisasi(str_text):
  for i in norm:
    str_text = str_text.replace(i, norm[i])
  return str_text

df['Review'] = df['Review'].apply(lambda x: normalisasi(x))
df

,Produk,Nama,Waktu,Review,Rating
0,Kaos polo polos / Kaos kerah / Kaos wangki,K***i,1 hari lalu,ukurannya pas tinggi badan 160 - 170 cm,5
1,Kaos pria o neck need coffe,Veronica,1 hari lalu,"sesuai foto, cuma tipis aja.. kl difoto kliata...",5
2,Croptop Wanita girl - Lengan Pendek Cotton Com...,FVou,2 hari lalu,155/48. i love it!,5
3,Kaos Polos Wanita Cotton Combed 30s,A***i,2 hari lalu,ukurannya pas tinggi badan 161 - 170 cm berat ...,5
4,Kaos Polos Wanita Cotton Combed 30s,nia,2 hari lalu,ukurannya pas,5
...,...,...,...,...,...
372,KAOS POLO COMBED,iwan,9 bulan lalu,"barang nya bagus, sablon ny keren bahan nya ad...",5
373,KAOS 78th DIRGAHAYU REPUBLIK INDONESIA / KAOS ...,alfira,9 bulan lalu,ukurannya pas,5
374,KAOS 78th DIRGAHAYU REPUBLIK INDONESIA / KAOS ...,Yulina,9 bulan lalu,terima kasih \n\nterima kasih,5
375,Kaos wanita friends - Cotton Combed 30s,Meutia,9 bulan lalu,bajunya pas dan nggak nerawang. untukuk harga ...,5


In [ ]:
#Stopword
more_stop_words = ["tidak", "yang", "dan", "di"]

stop_words = StopWordRemoverFactory().get_stop_words()
stop_words.extend(more_stop_words)

new_array = ArrayDictionary(stop_words)
stop_words_remover_new = StopWordRemover(new_array)

def stopword(str_text):
  str_text = stop_words_remover_new.remove(str_text)
  return str_text

df['Review'] = df['Review'].apply(lambda x: stopword(x))
df

,Produk,Nama,Waktu,Review,Rating
0,Kaos polo polos / Kaos kerah / Kaos wangki,K***i,1 hari lalu,ukurannya pas tinggi badan 160 - 170 cm,5
1,Kaos pria o neck need coffe,Veronica,1 hari lalu,"sesuai foto, cuma tipis aja.. kl difoto kliata...",5
2,Croptop Wanita girl - Lengan Pendek Cotton Com...,FVou,2 hari lalu,155/48. i love it!,5
3,Kaos Polos Wanita Cotton Combed 30s,A***i,2 hari lalu,ukurannya pas tinggi badan 161 - 170 cm berat ...,5
4,Kaos Polos Wanita Cotton Combed 30s,nia,2 hari lalu,ukurannya pas,5
...,...,...,...,...,...
372,KAOS POLO COMBED,iwan,9 bulan lalu,"barang nya bagus, sablon ny keren bahan nya ad...",5
373,KAOS 78th DIRGAHAYU REPUBLIK INDONESIA / KAOS ...,alfira,9 bulan lalu,ukurannya pas,5
374,KAOS 78th DIRGAHAYU REPUBLIK INDONESIA / KAOS ...,Yulina,9 bulan lalu,terima kasih \n\nterima kasih,5
375,Kaos wanita friends - Cotton Combed 30s,Meutia,9 bulan lalu,bajunya pas nggak nerawang. untukuk harga segi...,5


In [ ]:
#Tokenize (memisah tiap kata untuk klasifikasi)
tokenized = df['Review'].apply(lambda x:x.split())
tokenized

0       [ukurannya, pas, tinggi, badan, 160, -, 170, cm]
1      [sesuai, foto,, cuma, tipis, aja.., kl, difoto...
2                                [155/48., i, love, it!]
3      [ukurannya, pas, tinggi, badan, 161, -, 170, c...
4                                       [ukurannya, pas]
                             ...                        
372    [barang, nya, bagus,, sablon, ny, keren, bahan...
373                                     [ukurannya, pas]
374                       [terima, kasih, terima, kasih]
375    [bajunya, pas, nggak, nerawang., untukuk, harg...
376    [ukurannya, pas, tinggi, badan, <, 160, cm, be...
Name: Review, Length: 377, dtype: object

In [ ]:
#Stemming
def stemming(text_cleaning):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  do = []
  for w in text_cleaning:
    dt = stemmer.stem(w)
    do.append(dt)
  d_clean = []
  d_clean = " ".join(do)
  print(d_clean)
  return d_clean

tokenized = tokenized.apply(stemming)
tokenized.to_csv("/content/drive/MyDrive/Project1/JulClothing/stemmingJul.csv", index=False)

ukur pas tinggi badan 160 - 170 cm
sesuai foto cuma tipis aja kl foto kliatan bahan tebal tp sesuai harga lah
155 48 i love it
ukur pas tinggi badan 161 - 170 cm berat badan  50 kg
ukur pas
kaos lembuttt bahanya
bahan lumayan bagus
ukur pas
ukur terlalu kecil baik biasa pake xl naik jadi xxl bahan bagus sesuai harga kirim cepat
ukur pas
ukur terlalu kecil banget banget
bahan ok pas badan
ukur pas tinggi badan 150 - 160 cm berat badan 50 - 60 kg
ukur pas tinggi badan 150 - 160 cm berat badan 50 - 60 kg
sangat padan harga segitu kain bagus coba moga nyaman

ukur pas
baguuuusss adem
ukur pas
kirim cepat bahan nyata ga setip ekspektasi terima kasih
ukur pas nyaman pakai
bagus ukur kecil
ukur pas mantap baik sesuai pesan
barang sampai terimakasih gan
don t do this it s not your job don t sell or sell goods properly i thariew away the t-shirt you sent
good barang pas bahan nyaman
ukur bagi tangan pendek banget
bagus keren mantap tipisss ukur pas
ukur pas
pas banget murmer ukur pas berat bada

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Project1/JulClothing/stemmingJul.csv')
df1

,Review
0,ukur pas tinggi badan 160 - 170 cm
1,sesuai foto cuma tipis aja kl foto kliatan bah...
2,155 48 i love it
3,ukur pas tinggi badan 161 - 170 cm berat badan...
4,ukur pas
...,...
372,barang nya bagus sablon ny keren bahan nya ade...
373,ukur pas
374,terima kasih terima kasih
375,baju pas nggak nerawang untukuk harga gin udah...


In [ ]:
def convert_eng(review):
  translator = Translator(to_lang="en", from_lang="id")
  translation = translator.translate(review)
  return translation

df1['Review'] = df1['Review'].astype(str)
df1['Review_English'] = df1['Review'].apply(convert_eng)
df1.to_csv('/content/drive/MyDrive/Project1/JulClothing/translateJul.csv', index = False)

# Labelling

In [ ]:
data1 = pd.read_csv('/content/drive/MyDrive/Project1/JulClothing/translateJul.csv')
data1

,Review,Review_English
0,ukur pas tinggi badan 160 - 170 cm,fit measure height 160 - 170 cm
1,sesuai foto cuma tipis aja kl foto kliatan bah...,according to the photo is only thin if the pho...
2,155 48 i love it,155 48 i love it
3,ukur pas tinggi badan 161 - 170 cm berat badan...,fit measure height 161 - 170 cm body weight 5...
4,ukur pas,measure fit
...,...,...
372,barang nya bagus sablon ny keren bahan nya ade...,MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE ...
373,ukur pas,MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE ...
374,terima kasih terima kasih,MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE ...
375,baju pas nggak nerawang untukuk harga gin udah...,MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE ...


In [ ]:
data1['Review'] = data1['Review'].astype(str)
data1['Review_English'] = data1['Review_English'].astype(str)

In [ ]:
data_review = list(data1['Review_English'])
polaritas = 0

status = []
total_positif = total_negatif = total_netral = total = 0

for i, review in enumerate(data_review):
  analysis = TextBlob(review)
  polaritas += analysis.polarity

  if analysis.sentiment.polarity > 0.0:
    total_positif += 1
    status.append("Positif")
  elif analysis.sentiment.polarity == 0.0:
    total_netral += 1
    status.append("Netral")
  else:
    total_negatif += 1
    status.append("Negatif")

  total += 1

print(f'Hasil Analisis Data:\nPositif = {total_positif}\nNetral = {total_netral}\nNegatif = {total_negatif}')
print(f'\nTotal Data {total}')

Hasil Analisis Data:
Positif = 255
Netral = 58
Negatif = 64

Total Data 377


In [ ]:
data1['Klasifikasi'] = status
data1

,Review,Review_English,Klasifikasi
0,ukur pas tinggi badan 160 - 170 cm,fit measure height 160 - 170 cm,Positif
1,sesuai foto cuma tipis aja kl foto kliatan bah...,according to the photo is only thin if the pho...,Negatif
2,155 48 i love it,155 48 i love it,Positif
3,ukur pas tinggi badan 161 - 170 cm berat badan...,fit measure height 161 - 170 cm body weight 5...,Positif
4,ukur pas,measure fit,Positif
...,...,...,...
372,barang nya bagus sablon ny keren bahan nya ade...,MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE ...,Positif
373,ukur pas,MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE ...,Positif
374,terima kasih terima kasih,MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE ...,Positif
375,baju pas nggak nerawang untukuk harga gin udah...,MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE ...,Positif
